In [1]:
import pandas as pd
import os
import re
import romkan
import unicodedata

In [135]:
# get irregular words from ../data/irregular/irregular.csv and create a dict to avoid those words

In [135]:
vocab = pd.read_csv('../data/created/JMdict_e.csv', sep='\t')
kanji = pd.read_csv('../data/created/kanjidic2.csv')

In [136]:
# Run once
del vocab['pos']
del vocab['meaning']
del kanji['Unnamed: 0']
vocab.pronunciation = vocab.pronunciation.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
kanji.on = kanji.on.map(lambda x: re.findall("""[\u3040-\u309f]+""", romkan.to_hiragana(romkan.to_roma(x))))
##############################
#kanji.kun = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x))))) # THIS THREW AWAY GOOD INFO
##############################
kanji['kun1'] = kanji.kun.map(lambda x: list(set(map(lambda y: y.split('.')[0],re.findall("""[\u3040-\u309f.]+""", x)))))
kanji['kun2'] = kanji.kun.map(lambda x: list(set(map(lambda y: ''.join(y.split('.')),re.findall("""[\u3040-\u309f.]+""", x)))))
# I should also include the kun without splitting somehow... aka shi.ku = shi and shiku
kanji.nanori = kanji.nanori.map(lambda x: re.findall("""[\u3040-\u309f]+""", x))
kanji.meaning = kanji.meaning.map(lambda x: re.findall("""[a-zA-Z-]+ *[a-zA-Z-]+""", x))
#char_ser = pd.Series(kanji.on.values,index=kanji.character).to_dict()
kanji['onkun'] = kanji.kun1 + kanji.kun2 + kanji.on
#kanji.onkun.map(lambda x: list(set(x))) ########## NOW MAKE SURE ALL ARE SET TO LISTS SO THERE ARE NO DUPLICATES
char_ser = pd.Series(kanji.onkun.values,index=kanji.character).to_dict()
kanji.head()

,character,freq,meaning,on,kun,nanori,kun1,kun2,onkun
0,亜,1509.0,"[Asia, rank next, come after, -ous]",[あ],['つ.ぐ'],"[や, つぎ, つぐ]",[つ],[つぐ],"[つ, つぐ, あ]"
1,唖,NaN,"[mute, dumb]","[あ, あく]",['おし'],[],[おし],[おし],"[おし, おし, あ, あく]"
2,娃,NaN,[beautiful],"[あ, あい, わ]",['うつく.しい'],[い],[うつく],[うつくしい],"[うつく, うつくしい, あ, あい, わ]"
3,阿,1126.0,"[Africa, flatter, fawn upon, corner, nook, rec...","[あ, お]","['おもね.る', 'くま']","[ほとり, あず, あわ, おか, きた, な]","[おもね, くま]","[くま, おもねる]","[おもね, くま, くま, おもねる, あ, お]"
4,哀,1715.0,"[pathetic, grief, sorrow, pathos, pity, sympat...",[あい],"['あわ.れ', 'あわ.れむ', 'かな.しい']",[],"[あわ, かな]","[あわれむ, あわれ, かなしい]","[あわ, かな, あわれむ, あわれ, かなしい, あい]"


In [148]:
#vocab.vocab = vocab.vocab.map(lambda x: romkan.to_hiragana(romkan.to_roma(x)))
#vocab.head()

#char_ser['亜']
len('boy'[3:])

#print(jlpt[jlpt.Kanji.str.contains('風')])
#print(char_ser['風'])

0

In [20]:
irregular = pd.read_csv('../data/irregular/irregular.csv')
jlpt = pd.read_csv('../data/vocab/jlptvocab.csv')

In [82]:
jlpt.tail()

,Kanji,Hiragana,English
8318,忘れ物,わすれもの,lost article
8319,笑う,わらう,"to laugh,to smile"
8320,割合,わりあい,"rate,ratio,percentage"
8321,割れる,われる,to break
8337,消しゴム,けしゴム,"eraser, rubber"


In [77]:
def unvoice(char):
    if not ('\u3041' <= char <= '\u3096') or len(char) > 1 or not unicodedata.decomposition(char):
        return char
    return unicodedata.normalize('NFD', char)[0]

In [ ]:
def check_word(word,cur,pron,pastprons,pastpron):
    if cur > len(word):
        return pron
    try:
        prons = set(char_ser[word[cur]])
    except:
        print(word, ' is not in the dictionary')
    
    past_prons.append('')

In [153]:
def recurs(kanji, word, pron_list, done = False):
    if len(pron_list) == len(kanji):
        return None, False #pron_list?
    cur = kanji[len(pron_list)]
    if re.match("""[\u3040-\u30ff]""", cur) != None and cur == word[0]: #should we romkan to hiragana?? ALSO HYPHEN
        prons = cur
    else:
        prons = char_ser[cur] # .sort(key=len, reverse=True)
    for pron in prons:
        wordmatch = word[:len(pron)]
        wordmatch = unvoice(wordmatch[0]) + wordmatch[1:]
        if pron == wordmatch:
            pron_list.append(word[:len(pron)])
            word = word[len(pron):]
            if len(pron_list) == len(kanji) and len(word) == 0: ## I didn't check for if I end too soon
                return pron_list, True
            new_pron_list, done = recurs(kanji,word,pron_list,done)
            if done:
                return new_pron_list, done
            word = pron_list.pop() + word
    return None, False

# update with unvoice the first syllable of word


#for _,line in jlpt.iterrows():
#    word = line.Kanji
#    cur = 0
#    pron = line.Hiragana
#    prons = list()
#    check_word(word,cur,pron,prons,'')
#    break
#for a in jlpt.iterrows():
#    print(a)

In [154]:
#char_ser['車']
recurs('風呂敷','ふろしき',[])
#char_ser['敷']#.sort(key=len, reverse=True)    # EITHER INCLUDE NANORI OR FIX THE ABOVE

(['ふ', 'ろ', 'しき'], True)

In [43]:
#jlpt[jlpt.Kanji.str.contains('亜')]
#jlpt[jlpt.Kanji.isnull()]
jlpt = jlpt[jlpt.Kanji.isnull() == 0]# and jlpt.Hiragana.isnull() == 0 and jlpt.English.isnull() == 0]
jlpt = jlpt[jlpt.Hiragana.isnull() == 0]
jlpt = jlpt[jlpt.English.isnull() == 0]
#print(jlpt)

In [45]:
char_on = pd.Series(kanji.on.values,index=kanji.character).to_dict()
char_kun = pd.Series(kanji.kun.values,index=kanji.character).to_dict()

In [96]:
#jlpt[jlpt.Kanji.str.contains('亜')]
jlpt.head()

,Kanji,Hiragana,English
0,嗚呼,ああ,"Ah!, Oh!, Alas!"
1,相,あい,"together, mutually, fellow"
2,相変わらず,あいかわらず,"as ever, as usual, the same"
3,愛想,あいそ,"civility, courtesy, compliments, sociability, ..."
4,相対,あいたい,"confrontation, facing, between ourselves, no t..."


In [81]:
#vocab.vocab kanji.character[0]

#vocab[vocab.vocab.str.contains('亜')]
#irregular.head()#[irregular.Link == '印度尼西亜']
#print(vocab[vocab.vocab == '印度尼西亜'])
#print('印', char_ser['印'])
#print('度', char_ser['度'])
#print('尼', char_ser['尼'])
#print('西', char_ser['西'])
#print('亜', char_ser['亜'])

#print(kanji.kun[kanji.character == '尼'])

In [139]:
def kanjiru(index = 25):
    word = vocab.loc[index].vocab
    pron = vocab.loc[index].pronunciation
    print(word, pron)
    for ji in word:
        if re.match("""[\u3040-\u30ff]""",ji) != None:
            print(ji, ji)
            pron = pron[1:]
        else:
            pron_list = char_ser[ji]
            pron_list.sort(key=len, reverse=True)
            print(pron_list)
            for pr in pron_list:
                if pron[0:len(pr)] == pr: # re match beginning of string
                    print(ji, pr)
                    pron = pron[len(pr):]
# IF FAIL THEN TRY AGAIN FROM LESS PRON. IF FAIL AGAIN THEN TRY IN REVERSE. IF FAIL AGAIN...

In [92]:

word = '漢数字'
pron = 'かんすうじ'
for ji in word:
    pron_list = re.findall("""[\u3040-\u309f]+""", char_ser[ji])
    pron_list.sort(key=len, reverse=True)
    print(pron_list)
    for pr in pron_list:
        if pron[0:len(pr)] == pr: # re match beginning of string
            print(ji, pr)
            pron = pron[len(pr):]
    #re.match()
    
#next match up each kanji to on or kun for exploratory analysis

#Maybe if word is only kanji - first check on list, if word is mixture - first check kun?

# Also put in checks for hirgana matching ()

['かん']
漢 かん
['すう', 'さく', 'そく', 'しゅ', 'す']
数 すう
['じ']
字 じ


In [152]:
#kanjiru(28)
#char_ser['呪']
print(kanji.kun[kanji.character =='御'])
print(kanji.kun[kanji.character =='襁'])
print(kanji.kun[kanji.character =='褓'])
print(kanji.kun[kanji.character =='気'])
print(kanji.kun[kanji.character =='触'])

821    [み, おん, お]
Name: kun, dtype: object
5494    [むつき]
Name: kun, dtype: object
5488    [むつき]
Name: kun, dtype: object
473    [いき]
Name: kun, dtype: object
1417    [さわ, ふ]
Name: kun, dtype: object


In [128]:
kanji[kanji.character == '明']

,character,freq,meaning,on,kun,nanori,onkun
2663,明,67.0,"[bright, light]","[みょう, めい, みん]","[あ, あか, あか, あか, あき, あ, あ, あ, あ, あ]","[あきら, あけ, あす, きら, け, さや, さやか, とし, はる, み, め]","[みょう, あか, あか, あか, あき, めい, みん, あ, あ, あ, あ, あ, あ]"


In [133]:
#print(kanji[kanji.character == '漢'][['on','kun']])
#print(kanji[kanji.character == '数'][['on','kun']])
#print(kanji[kanji.character == '字'][['on','kun']])
for a in kanji[kanji.character == '明'].kun:
    print(a)

['あ.かり', 'あか.るい', 'あか.るむ', 'あか.らむ', 'あき.らか', 'あ.ける', '-あ.け', 'あ.く', 'あ.くる', 'あ.かす']


In [3]:
#nounDic = pd.DataFrame(columns=['kanji','pronunciation','type','meaning'])
dir = os.path.dirname('''../data/created/JMdict_e.csv''')
if not os.path.exists(dir):
    os.makedirs(dir)

file = open('''../data/created/JMdict_e.csv''', 'w+')
file.truncate()
file.write('vocab\tpronunciation\tpos\tmeaning\n')

32

In [4]:
#i = 0
for noun in root.findall('entry'):
  word = noun.find('k_ele')
  if word is not None:
    word = word.find('keb').text
    pron = noun.find('r_ele').find('reb').text
    noun_child = noun.find('sense')
    noun_type = noun_child.find('pos').text
    meaning = noun_child.find('gloss').text
    file.write(word + '\t' + pron + '\t' + noun_type + '\t' + meaning + '\n')
    #if noun_child.find('pos') is not None:
    #  noun_type = noun_child.find('name_type').text
    #  meaning = noun_child.find('trans_det').text
    #  file.write(word + '\t' + pron + '\t' + noun_type + '\t' + meaning + '\n')
    #nounDic.loc[i] = [word,pron,noun_type,meaning] # taking all the time (probably)
    #i = i + 1
    
    #try to write directly to a CSV file (keep file open)

#kanji_list = []
#pron_list = []
#type_list = []
#meaning_list = []
#for noun in root.findall('entry'):
#  kanji = noun.find('k_ele')
#  if kanji is not None:
#    noun_child = noun.find('trans')
#    if noun_child is not None:
#      if noun.find('r_ele') is not None:
#        if noun_child.find('name_type') is not None:
#          kanji_list.append(kanji.find('keb').text)
#          pron_list.append(noun.find('r_ele').find('reb').text)
#          type_list.append(noun_child.find('name_type').text)
#          meaning_list.append(noun_child.find('trans_det').text)
#    nounDic.loc[i] = [kanji,pron,noun_type,meaning]
#    i = i + 1
#nounDic = pd.DataFrame({'kanji': kanji_list,'pronunciation': pron_list,'type': type_list,'meaning': meaning_list}, columns=['kanji','pronunciation','type','meaning'])

  #freq = noun.find('misc').find('freq') #check if none
  #if freq != None:
  #      freq = freq.text
  #character = noun.find('literal').text
  #noun_child = noun.find('reading_meaning')
  #if noun_child != None:
  #  noun_child = noun_child.find('rmgroup')
  #  if noun_child != None:
  #    meaning_list = noun_child.findall('meaning') #list of elements -> list of text
  #    meaning = [m.text if m.attrib == {} else None for m in meaning_list] #remove Nones now
  #    meaning = [m for m in meaning if m is not None]
  #    reading_list = noun_child.findall('reading')
  #    on = [o.text if o.attrib == {'r_type': 'ja_on'} else None for o in reading_list] #remove Nones now
  #    kun = [k.text if k.attrib == {'r_type': 'ja_kun'} else None for k in reading_list]
  #    on = [o for o in on if o is not None]
  #    kun = [k for k in kun if k is not None]
  #    nanori_list = noun.find('reading_meaning').findall('nanori')
  #    nanori = [a.text for a in nanori_list]
  #    nounDic.loc[i] = [character,freq,meaning,on,kun,nanori]
  #    i = i + 1

In [6]:
#nounDic.tail()

,kanji,pronunciation,type,meaning
306515,沼田町,ぬまたちょう,place name,Numatachou
306516,沼田町,ぬまたまち,place name,Numatamachi
306517,沼田町阿戸,ぬまたちょうあと,place name,Numatachouato
306518,沼田町吉山,ぬまたちょうよしやま,place name,Numatachouyoshiyama
306519,沼田町大塚,ぬまたちょうおおつか,place name,Numatachouootsuka


In [20]:
#temp = {}

In [22]:
#temp.update({'boy':1})

In [23]:
#temp

{'boy': 1}

In [5]:
file.close()

In [1]:
#import romkan

ImportError: No module named 'romkan'

In [6]:
# now create a dictionary of {word: meaning} and write to json file -- this is for personal website !!!!

In [11]:
all_words = pd.read_csv('../data/created/JMdict_e.csv', sep='\t')

In [50]:
words_dict = {}
for entry in all_words.iterrows():
    words_dict.update({entry[1][0]:entry[1][1] + '\t' + entry[1][2] + '\t' + entry[1][3]})

In [19]:
all_words.head()

,vocab,pronunciation,pos,meaning
0,〃,おなじ,noun (common) (futsuumeishi),ditto mark
1,仝,どうじょう,noun (common) (futsuumeishi),as above mark
2,々,くりかえし,noun (common) (futsuumeishi),repetition of kanji (sometimes voiced)
3,漢数字ゼロ,かんすうじゼロ,noun (common) (futsuumeishi),kanji zero
4,○,まる,noun (common) (futsuumeishi),circle (sometimes used for zero)


In [36]:
#next(all_words.iterrows())[2][2]

IndexError: tuple index out of range

In [51]:
import json

# load from file:
#with open('/path/to/my_file.json', 'r') as f:
#    try:
#        data = json.load(f)
#    # if the file is empty the ValueError will be thrown
#    except ValueError:
#        data = {}

# save to file:
with open('../word_dictionary.json', 'w') as f:
    json.dump(words_dict, f)
